<a href="https://colab.research.google.com/github/duytai800/Projects/blob/main/StrokePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib as plt
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


**Load dữ liệu vào, dataset là tập train , test là tập test**

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/tempoluxury/train.csv')
print(dataset)

         id  gender   age  hypertension  ...   smoking_status blood height stroke
0        77  Female   NaN           0.0  ...          Unknown     O    NaN      0
1        84    Male  55.0           0.0  ...     never smoked     B   1.59      0
2        91  Female  42.0           0.0  ...     never smoked     B    NaN      0
3        99  Female  31.0           0.0  ...          Unknown     O   1.24      0
4       121  Female  38.0           0.0  ...          Unknown     O    NaN      0
...     ...     ...   ...           ...  ...              ...   ...    ...    ...
3995  72867    Male  16.0           0.0  ...          Unknown     B    NaN      0
3996  72882    Male  47.0           0.0  ...  formerly smoked     B   1.79      0
3997  72911   Other  57.0           1.0  ...           smokes    AB   1.15      0
3998  72915  Female  45.0           0.0  ...  formerly smoked     O   1.33      0
3999  72918  Female  53.0           1.0  ...          Unknown     O   1.62      1

[4000 rows x 14

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/tempoluxury/public_test.csv')
print(test)

        id  gender   age  hypertension  ...   bmi   smoking_status blood height
0       67  Female  17.0           0.0  ...   NaN  formerly smoked    AB    NaN
1      210    Male  81.0           0.0  ...  31.4     never smoked     A   1.60
2      242    Male   4.0           NaN  ...  17.5          Unknown     O    NaN
3      711    Male  81.0           0.0  ...  22.2     never smoked    AB    NaN
4      724    Male  17.0           0.0  ...  44.7     never smoked     O   1.34
..     ...     ...   ...           ...  ...   ...              ...   ...    ...
495  72393  Female  23.0           0.0  ...  24.2     never smoked     A   1.82
496  72491    Male  53.0           0.0  ...  29.2           smokes     O   1.66
497  72562  Female  57.0           0.0  ...  32.8     never smoked    AB   1.56
498  72792  Female  53.0           NaN  ...   NaN     never smoked    AB   1.56
499  72940  Female   2.0           0.0  ...  17.6          Unknown    AB    NaN

[500 rows x 13 columns]


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 4000 non-null   int64  
 1   gender             4000 non-null   object 
 2   age                3893 non-null   float64
 3   hypertension       3770 non-null   float64
 4   heart_disease      3910 non-null   float64
 5   ever_married       4000 non-null   object 
 6   work_type          4000 non-null   object 
 7   Residence_type     4000 non-null   object 
 8   avg_glucose_level  3829 non-null   float64
 9   bmi                2884 non-null   float64
 10  smoking_status     4000 non-null   object 
 11  blood              3990 non-null   object 
 12  height             2928 non-null   float64
 13  stroke             4000 non-null   int64  
dtypes: float64(6), int64(2), object(6)
memory usage: 437.6+ KB


In [ ]:
dataset.drop(columns = "id", inplace = True)
dataset.drop(dataset[dataset.age < 30].index, inplace=True) #từ dữ liệu ta thấy hầu hết những người dưới 30 tuổi đều không bị đột quỵ nên ta bỏ đi để giảm số stroke 0


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2842 entries, 0 to 3999
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             2842 non-null   object 
 1   age                2735 non-null   float64
 2   hypertension       2671 non-null   float64
 3   heart_disease      2774 non-null   float64
 4   ever_married       2842 non-null   object 
 5   work_type          2842 non-null   object 
 6   Residence_type     2842 non-null   object 
 7   avg_glucose_level  2726 non-null   float64
 8   bmi                2042 non-null   float64
 9   smoking_status     2842 non-null   object 
 10  blood              2835 non-null   object 
 11  height             2396 non-null   float64
 12  stroke             2842 non-null   int64  
dtypes: float64(6), int64(1), object(6)
memory usage: 310.8+ KB


**Tiến hành thêm dữ liệu khuyết thiếu bằng hàm mean**

In [ ]:
dataset['age'].fillna(value=dataset['age'].mean(), inplace=True)
dataset['hypertension'].fillna(value=dataset['hypertension'].mean(), inplace=True)
dataset['heart_disease'].fillna(value=dataset['heart_disease'].mean(), inplace=True)
dataset['avg_glucose_level'].fillna(value=dataset['avg_glucose_level'].mean(), inplace=True)
dataset['bmi'].fillna(value=dataset['bmi'].mean(), inplace=True)
dataset['height'].fillna(value=dataset['height'].mean(), inplace=True)

In [ ]:
test['age'].fillna(value=test['age'].mean(), inplace=True)
test['hypertension'].fillna(value=test['hypertension'].mean(), inplace=True)
test['heart_disease'].fillna(value=test['heart_disease'].mean(), inplace=True)
test['avg_glucose_level'].fillna(value=test['avg_glucose_level'].mean(), inplace=True)
test['bmi'].fillna(value=test['bmi'].mean(), inplace=True)
test['height'].fillna(value=test['height'].mean(), inplace=True)

**encoder convert cái dữ liệu dạng chữ sang số**

In [ ]:
dataset['gender']=dataset['gender'].apply(lambda x : 1 if x=='Male' else (2 if x == "Female" else 0)) 
dataset['Residence_type']=dataset['Residence_type'].apply(lambda x : 1 if x=='Rural' else (2 if x == "Urban" else 0)) 
dataset['ever_married']=dataset['ever_married'].apply(lambda x : 1 if x=='Yes' else 0 )
dataset['work_type']=dataset['work_type'].apply(lambda x : 1 if x=='Private' else (0 if x == 'Self-employed' else (2 if x == 'Govt_job' else 3)))
dataset['smoking_status']=dataset['smoking_status'].apply(lambda x : 1 if x=='never smoked' else (2 if x == 'formerly smoked' else (3 if x == 'smokes' else 0)))
dataset['blood']=dataset['blood'].apply(lambda x : 0 if x =='O' else (3 if x == 'AB' else (1 if x == 'A' else 2)))

In [ ]:
print(dataset)

         id  gender   age  hypertension  ...  smoking_status  blood  height  stroke
0        77       0   NaN           0.0  ...               0      2     NaN       0
1        84       0  55.0           0.0  ...               0      2    1.59       0
2        91       0  42.0           0.0  ...               0      2     NaN       0
3        99       0  31.0           0.0  ...               0      2    1.24       0
4       121       0  38.0           0.0  ...               0      2     NaN       0
...     ...     ...   ...           ...  ...             ...    ...     ...     ...
3995  72867       0  16.0           0.0  ...               0      2     NaN       0
3996  72882       0  47.0           0.0  ...               0      2    1.79       0
3997  72911       0  57.0           1.0  ...               0      2    1.15       0
3998  72915       0  45.0           0.0  ...               0      2    1.33       0
3999  72918       0  53.0           1.0  ...               0      2    1.62 

In [ ]:
test['gender']=test['gender'].apply(lambda x : 1 if x=='Male' else (2 if x == "Female" else 0)) 
test['Residence_type']=test['Residence_type'].apply(lambda x : 1 if x=='Rural' else (2 if x == "Urban" else 0)) 
test['ever_married']=test['ever_married'].apply(lambda x : 1 if x=='Yes' else 0 )
test['work_type']=test['work_type'].apply(lambda x : 1 if x=='Private' else (0 if x == 'Self-employed' else (2 if x == 'Govt_job' else 3)))
test['smoking_status']=test['smoking_status'].apply(lambda x : 1 if x=='never smoked' else (2 if x == 'formerly smoked' else (3 if x == 'smokes' else 0)))
test['blood']=test['blood'].apply(lambda x : 0 if x =='O' else (3 if x == 'AB' else (1 if x == 'A' else 2)))

NameError: ignored

In [ ]:
print(dataset)

      gender        age  hypertension  ...  blood    height  stroke
0          2  54.888483           0.0  ...      0  1.602538       0
1          1  55.000000           0.0  ...      2  1.590000       0
2          2  42.000000           0.0  ...      2  1.602538       0
3          2  31.000000           0.0  ...      0  1.240000       0
4          2  38.000000           0.0  ...      0  1.602538       0
...      ...        ...           ...  ...    ...       ...     ...
3994       2  52.000000           0.0  ...      2  1.602538       0
3996       1  47.000000           0.0  ...      2  1.790000       0
3997       0  57.000000           1.0  ...      3  1.150000       0
3998       2  45.000000           0.0  ...      0  1.330000       0
3999       2  53.000000           1.0  ...      0  1.620000       1

[2842 rows x 13 columns]


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2842 entries, 0 to 3999
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             2842 non-null   int64  
 1   age                2842 non-null   float64
 2   hypertension       2842 non-null   float64
 3   heart_disease      2842 non-null   float64
 4   ever_married       2842 non-null   int64  
 5   work_type          2842 non-null   int64  
 6   Residence_type     2842 non-null   int64  
 7   avg_glucose_level  2842 non-null   float64
 8   bmi                2842 non-null   float64
 9   smoking_status     2842 non-null   int64  
 10  blood              2842 non-null   int64  
 11  height             2842 non-null   float64
 12  stroke             2842 non-null   int64  
dtypes: float64(6), int64(7)
memory usage: 310.8 KB


In [ ]:
dataset.corr()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,blood,height,stroke
gender,1.000000,-0.043413,-0.050026,-0.105546,-0.042193,-0.002770,-0.078008,-0.086929,-0.031532,-0.048620,-0.122948,-0.014641,-0.015984
age,-0.043413,1.000000,0.189179,0.268536,0.136300,-0.186960,0.038517,0.193458,-0.087980,-0.033325,0.009123,0.101932,0.181198
hypertension,-0.050026,0.189179,1.000000,0.069789,0.042842,-0.069622,0.010408,0.136864,0.079895,0.043895,0.005858,-0.066977,0.058874
heart_disease,-0.105546,0.268536,0.069789,1.000000,0.014612,-0.070113,-0.008107,0.144211,-0.026581,0.030298,0.023062,0.036388,0.091268
ever_married,-0.042193,0.136300,0.042842,0.014612,1.000000,-0.058555,0.023681,0.060480,0.046855,0.026452,-0.006664,-0.005795,0.006084
work_type,-0.002770,-0.186960,-0.069622,-0.070113,-0.058555,1.000000,-0.035085,-0.034553,-0.047091,-0.025775,-0.018800,0.004964,-0.009046
Residence_type,-0.078008,0.038517,0.010408,-0.008107,0.023681,-0.035085,1.000000,0.004026,0.001605,-0.009943,-0.050939,0.013235,0.190412
avg_glucose_level,-0.086929,0.193458,0.136864,0.144211,0.060480,-0.034553,0.004026,1.000000,0.136413,0.042456,0.016953,-0.144194,0.098175
bmi,-0.031532,-0.087980,0.079895,-0.026581,0.046855,-0.047091,0.001605,0.136413,1.000000,0.054071,-0.018500,-0.736718,-0.018039
smoking_status,-0.048620,-0.033325,0.043895,0.030298,0.026452,-0.025775,-0.009943,0.042456,0.054071,1.000000,-0.004435,-0.016252,0.001839


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 500 non-null    int64  
 1   gender             500 non-null    int64  
 2   age                500 non-null    float64
 3   hypertension       500 non-null    float64
 4   heart_disease      500 non-null    float64
 5   ever_married       500 non-null    int64  
 6   work_type          500 non-null    int64  
 7   Residence_type     500 non-null    int64  
 8   avg_glucose_level  500 non-null    float64
 9   bmi                500 non-null    float64
 10  smoking_status     500 non-null    int64  
 11  blood              500 non-null    int64  
 12  height             500 non-null    float64
dtypes: float64(6), int64(7)
memory usage: 50.9 KB


In [ ]:
print(test)

        id  gender   age  ...  smoking_status  blood    height
0       67       2  17.0  ...               2      3  1.620718
1      210       1  81.0  ...               1      1  1.600000
2      242       1   4.0  ...               0      0  1.620718
3      711       1  81.0  ...               1      3  1.620718
4      724       1  17.0  ...               1      0  1.340000
..     ...     ...   ...  ...             ...    ...       ...
495  72393       2  23.0  ...               1      1  1.820000
496  72491       1  53.0  ...               3      0  1.660000
497  72562       2  57.0  ...               1      3  1.560000
498  72792       2  53.0  ...               1      3  1.560000
499  72940       2   2.0  ...               0      3  1.620718

[500 rows x 13 columns]


**tiến hành loại bỏ 2 cột hypertension và heart_disease**

In [ ]:
dataset.drop(columns='hypertension', inplace=True)
dataset.drop(columns='heart_disease', inplace = True)

KeyError: ignored

In [ ]:
test.drop(columns='hypertension', inplace=True)
test.drop(columns='heart_disease', inplace = True)

In [ ]:
dataset.drop(columns='smoking_status', inplace=True)
test.drop(columns='smoking_status', inplace = True)

**kiểm tra số cột còn lại sau khi loại bỏ**

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2842 entries, 0 to 3999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             2842 non-null   int64  
 1   age                2842 non-null   float64
 2   ever_married       2842 non-null   int64  
 3   work_type          2842 non-null   int64  
 4   Residence_type     2842 non-null   int64  
 5   avg_glucose_level  2842 non-null   float64
 6   bmi                2842 non-null   float64
 7   blood              2842 non-null   int64  
 8   height             2842 non-null   float64
 9   stroke             2842 non-null   int64  
dtypes: float64(4), int64(6)
memory usage: 244.2 KB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 500 non-null    int64  
 1   gender             500 non-null    int64  
 2   age                500 non-null    float64
 3   ever_married       500 non-null    int64  
 4   work_type          500 non-null    int64  
 5   Residence_type     500 non-null    int64  
 6   avg_glucose_level  500 non-null    float64
 7   bmi                500 non-null    float64
 8   blood              500 non-null    int64  
 9   height             500 non-null    float64
dtypes: float64(4), int64(6)
memory usage: 39.2 KB


**chia tập train và test**

In [ ]:
x_train = dataset.iloc[:, :9].values

y_train = dataset.iloc[:, 9:].values

x_test = test.iloc[:,1:].values

res_id = np.array(test.id)



**xử lí mất cân bằng**

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

ud = RandomUnderSampler(sampling_strategy=0.25)

x_train, y_train= ud.fit_sample(x_train, y_train) # dùng hàm random để lấy ra 1 số dữ liệu stroke 0

print("after OverSampling, counts of label '1': {}".format(sum(y_train==1)))
print("after OverSampling, counts of label '0': {} \n".format(sum(y_train==0)))
x_train.shape

after OverSampling, counts of label '1': [130]
after OverSampling, counts of label '0': [520] 



/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


(650, 9)

In [ ]:
from imblearn.over_sampling import SMOTE


sm = SMOTE(random_state=2)#tăng số stroke 1 lên cho cân bằng

x_train, y_train = sm.fit_resample(x_train, y_train.ravel())


print("After OverSampling, counts of label '1': {}".format(sum(y_train==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train==0)))

After OverSampling, counts of label '1': 520
After OverSampling, counts of label '0': 520


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
x_train.shape

(2600, 10)

**train model**

In [ ]:
from sklearn.ensemble import RandomForestClassifier #dung thuat toan ramdon forest



In [ ]:
clf = RandomForestClassifier()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,

In [ ]:
submission = pd.DataFrame(zip(res_id,y_pred), columns = ['id','stroke'])
submission


,id,stroke
0,67,0
1,210,0
2,242,0
3,711,0
4,724,0
...,...,...
495,72393,0
496,72491,1
497,72562,0
498,72792,0


In [ ]:
df = pd.DataFrame(submission)
submit = df.to_csv (r'/content/drive/MyDrive/tempoluxury/sub.csv', index = False, header=True)

**Train MOdel with XGB**

In [ ]:
'''clf =XGBClassifier()
clf.fit(x_train, y_train)
y_pred1 = clf.predict(x_test)
submission = pd.DataFrame(zip(res_id,y_pred1), columns = ['id','stroke'])
submission
df = pd.DataFrame(submission)
submit = df.to_csv (r'/content/drive/MyDrive/tempoluxury/sub.csv', index = False, header=True)

In [ ]:
'''y_pred1

array([0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,